# Introdução
## Contexto do projeto
Todos os anos letivos, a equipa administrativa do **IPCA** enfrenta dificuldades na criação dos horários das aulas, devido às restrições complexas relacionadas com **professores, cursos, horários e salas disponíveis**. Este é um problema típico de **Satisfação de Restrições (*Constraint Satisfaction Problem* - CSP)**, onde é necessário encontrar soluções que satisfaçam um conjunto de *hard constraints* e *soft constraints*.

## Objetivo do projeto
O principal objetivo do projeto é criar um sistema que produza horários **válidos, otimizados e que respeitem todas as restrições**.

## Equipa de desenvolvimento
- Duarte Pereira — Nº 27959
- Hugo Especial — Nº 27963
- Paulo Gonçalves — Nº 27966 
- Marco Cardoso — Nº 27969 
- Hugo Pereira — Nº 27970

---

# *Design* do agente

## Formulação do problema como um Problema de Satisfação de Restrições (CSP)
Formular um problema de CSP exige identificar claramente três elementos:
- Variáveis, domínios e restrições.

E o objetivo é encontrar uma atribuição de valores que satisfaça todas as restrições.

## Definição de variáveis, domínio e restrições
### **Variáveis**:
$X = \{ Sala, Dia, Hora \}$

### **Domínio**:  
- $D(Sala) = \{ (Room A, Room C, Room G, Lab02) \}$
- $D(Dia) = \{ (Segunda, Terça, Quarta, Quinta, Sexta) \}$
- $D(Hora) = \{ (9h00 - 11h00, 11h00 - 13h00, 14h00 - 16h00, 16h00 - 18h00) \}$

### **Restrições**:
Para facilitar a formulação do problema, as restrições foram divididas em categorias.

#### Restrições temporais:

- Cada aula dura **2 horas**.
```python
def check_duration(aula): 
```

# Hard

- Uma turma **não pode ter mais de 3 aulas por dia**. 
```python
def max_three_per_day_turma(*aulas):
```

- Cada curso pode ter **1 ou 2 aulas por semana**. 
```python
def exactly_two_per_uc(*aulas, ucs=None):
```

- Todas as turmas têm **10 aulas semanais**. 
``` python
def exactly_ten_per_turma(*aulas, turmas=None):
```

# Soft
- As aulas de cada dia devem ser **consecutivas**.  
``` python
def check_consecutive_classes(*aulas):
```

- Aulas da mesma unidade curricular devem ocorrer em **dias distintos**. 
```python
def check_distinct_day_classes(*aulas): 
```

- Cada turma deve ter, se possível, **apenas 4 dias de aulas por semana**. 
```python
def check_weekly_days(*aulas):
```

#### Restrições de professores:
- O horário deve respeitar a **disponibilidade dos professores**. 
```python
def check_professor_availability(prof_id):
```

#### Restrições de salas:
- O número de **salas diferentes por turma** deve ser minimizado. 
```python
def check_different_classes(*aulas):
```

### Importação dos dados
Os dados foram importados a partir de ficheiros CSV (*Comma Separated Value*), utilizando Python. Cada ficheiro contém informações sobre diferentes entidades (cursos, professores, salas, turmas, unidades curriculares e disponibilidades.)

In [ ]:
import csv

def load_csv(path):
    data = []
    with open(path, mode='r', encoding='utf-8') as f:
        reader = csv.DictReader(f)
        for row in reader:
            data.append(row)
    return data

def load_courses(path='dados/cursos.csv'):
    cursos = load_csv(path)
    for c in cursos:
        c['id'] = int(c['id'])
        c['n_turmas'] = int(c['n_turmas'])
    return cursos

def load_teachers(path='dados/professores.csv'):
    professores = load_csv(path)
    for p in professores:
        p['id'] = int(p['id'])
    return professores

def load_rooms(path='dados/salas.csv'):
    salas = load_csv(path)
    for s in salas:
        s['id'] = int(s['id'])
    return salas


def load_curricular_units(path='dados/unidades_curriculares.csv'):
    ucs = load_csv(path)
    seen = set()
    ucs_unique = []
    for u in ucs:
        if u['id'] not in seen:
            u['id'] = int(u['id'])
            u['curso_id'] = int(u['curso_id'])
            u['n_aulas_semana'] = int(u['n_aulas_semana'])
            ucs_unique.append(u)
            seen.add(u['id'])
    return ucs_unique


def load_availabilities(path='dados/disponibilidades.csv'):
    disp = load_csv(path)
    for d in disp:
        d['prof_id'] = int(d['prof_id'])
        d['hora_inicio'] = int(d['hora_inicio'])
        d['hora_fim'] = int(d['hora_fim'])
    return disp

def load_classes(path='dados/turmas.csv'):
    turmas = load_csv(path)
    seen = set()
    turmas_unique = []
    for t in turmas:
        if t['id'] not in seen:
            t['id'] = int(t['id'])
            t['curso_id'] = int(t['curso_id'])
            turmas_unique.append(t)
            seen.add(t['id'])
    return turmas_unique


def load_all():
    return {
        'cursos': load_courses(),
        'professores': load_teachers(),
        'salas': load_rooms(),
        'unidades_curriculares': load_curricular_units(),
        'turmas': load_classes(),
        'disponibilidades': load_availabilities()
    }

if __name__ == "__main__":
    dados = load_all()
    print("=== Cursos ===")
    for c in dados['cursos']:
        print(c)

    print("\n=== Professores ===")
    for p in dados['professores']:
        print(p)

    print("\n=== Salas ===")
    for s in [s['nome'] for s in dados['salas']]:
        print(s)

    print("\n=== Turmas ===")
    for t in dados['turmas']:
        print(t)

    print("\n=== Unidades Curriculares ===")
    for u in dados['unidades_curriculares']:
        print(u)

    print("\n=== Disponibilidades ===")
    for d in dados['disponibilidades']:
        print(d)


## Função heurística para implementar as *soft constraints*


O processo de geração de horários é dividido em duas fases.
Na primeira, **o *solver* da biblioteca *constraint* gera todas as soluções válidas que respeitam as *hard constraints***, garantindo **apenas a validade das restrições obrigatórias**.

Na segunda, **as soluções são avaliadas e ordenadas através de uma função heurística**, que calcula uma **pontuação com base nas *soft constraints***, refletindo o grau de otimização das preferências.

A função heurística calcula uma pontuação para cada solução, considerando critérios como:
- o número de dias distintos que uma UC ocupa.
- se uma turma tiver 4 ou menos dias de aulas.
- se as aulas de um dia forem consecutivas.
- o menor número de salas diferentes.

A pontuação total da solução é a **soma dos valores obtidos em cada uma destas preferências**.

As **soluções com maior pontuação são consideradas ótimas**, uma vez que cumprem melhor as preferências definidas.

---


In [ ]:
# Uma UC deve ocupar o menor número de dias distintos possível (idealmente 2)
def check_distinct_day_classes(*aulas):
    uc_days = {}
    for aula in aulas:
        dia, _, _, _, _, uc = aula
        uc_days.setdefault(uc, set()).add(dia)
    # penaliza UC com mais de 2 dias distintos
    return sum(len(dias) - 2 for dias in uc_days.values() if len(dias) > 2) == 0


# Uma turma deve ter aulas em no máximo 4 dias por semana
def check_weekly_days(*aulas):
    turma_days = {}
    for aula in aulas:
        dia, _, _, _, turma, _ = aula
        turma_days.setdefault(turma, set()).add(dia)
    return all(len(dias) <= 4 for dias in turma_days.values())


# As aulas num mesmo dia devem ser consecutivas (sem buracos entre horas)
def check_consecutive_classes(*aulas):
    dia_to_horas_turma = {}
    for aula in aulas:
        dia, hora, _, _, turma, _ = aula
        dia_to_horas_turma.setdefault((turma, dia), []).append(hora)

    def hora_to_int(h):
        # aceita "09:00", "9:00" ou inteiro 9
        if isinstance(h, int):
            return h * 60  # assume blocos de 1h
        elif isinstance(h, str):
            partes = h.split(':')
            if len(partes) == 2:
                hh, mm = map(int, partes)
                return hh * 60 + mm
            else:
                return int(h) * 60
        else:
            raise ValueError(f"Formato de hora inesperado: {h}")

    for (turma, dia), horas in dia_to_horas_turma.items():
        horas_ordenadas = sorted(hora_to_int(h) for h in horas)
        # verifica se são consecutivas (intervalo de 60 min)
        for i in range(1, len(horas_ordenadas)):
            if horas_ordenadas[i] - horas_ordenadas[i-1] != 60:
                return False
    return True



# Uma turma deve usar o menor número de salas diferentes
def check_different_classes(*aulas):
    turma_salas = {}
    for aula in aulas:
        _, _, sala, _, turma, _ = aula
        turma_salas.setdefault(turma, set()).add(sala)
    # idealmente cada turma teria poucas salas — aqui verificamos <= 3
    return all(len(salas) <= 3 for salas in turma_salas.values())


## Análise Crítica

O agente consegue gerar horários válidos que respeitam todas as restrições obrigatórias e optimiza preferências definidas.

### Pontos positivos:
- Soluções válidas respeitam todas as hard constraints.
- A heurística permite identificar a solução com melhor pontuação, considerando:
  - Dias distintos por UC
  - Máximo de 4 dias de aulas por turma
  - Aulas consecutivas
  - Minimização de salas diferentes

### Limitações:
- O número de soluções cresce rapidamente com datasets maiores, tornando a execução mais lenta.
- A heurística não garante encontrar a solução globalmente óptima.
- Conflitos entre disponibilidade de professores e salas podem tornar algumas combinações inviáveis.

### Possível melhoria:
- Usar backtracking com heurísticas mais avançadas.
- Implementação de horário online.


## Conclusão

O projeto permitiu desenvolver um agente capaz de gerar horários de aulas que respeitam todas as restrições obrigatórias e otimizam preferências desejáveis.

Permitiu compreender a formulação de CSPs, implementação prática de hard e soft constraints, utilização do Jupyter Notebook como ferramenta de documentação.

O agente pode ser melhorado para lidar com datasets maiores ou restrições adicionais, mas já fornece uma solução funcional e automatizada para a geração de horários acadêmicos no IPCA.
